# Week 1 - Getting Started with tools and data

In [1]:
import pandas as pd
import numpy as np

tickerList = [
    "aapl.us",
    "amzn.us",
    "googl.us",
    "msft.us",
    "tsla.us",
]  # list of stocks to analyze

## Importing dataset into MultiIndexed dataframe

In [2]:
def load_data(tList):
    l = []
    for ticker in tList:
        l.append(
            pd.read_csv(f"stocks/{ticker}.csv", header=0)
        )  # read CSVs of different stocks
        l[-1].set_index("Date", inplace=True)  # convert date into a index (inner index)

    df = pd.concat(l, keys=tList, axis=0)  # concat with outer index being the ticker
    return df

In [3]:
df = load_data(tickerList)
df

Open       High        Low      Close    Volume  \
        Date                                                               
aapl.us 1984-09-07    0.42388    0.42902    0.41874    0.42388  23220030   
        1984-09-10    0.42388    0.42516    0.41366    0.42134  18022532   
        1984-09-11    0.42516    0.43668    0.42516    0.42902  42498199   
        1984-09-12    0.42902    0.43157    0.41618    0.41618  37125801   
        1984-09-13    0.43927    0.44052    0.43927    0.43927  57822062   
...                       ...        ...        ...        ...       ...   
tsla.us 2017-11-06  307.00000  307.50000  299.01000  302.78000   6482486   
        2017-11-07  301.02000  306.50000  300.03000  306.05000   5286320   
        2017-11-08  305.50000  306.89000  301.30000  304.31000   4725510   
        2017-11-09  302.50000  304.46000  296.30000  302.99000   5440335   
        2017-11-10  302.50000  308.36000  301.85000  302.99000   4621912   

                    OpenInt  
        Date                 
aapl.us 1984-09-07        0  
        1984-09-10        0  
        1984-09-11        0  
        1984-09-12        0  
        1984-09-13        0  
...                     ...  
tsla.us 2017-11-06        0  
        2017-11-07        0  
        2017-11-08        0  
        2017-11-09        0  
        2017-11-10        0  

[26691 rows x 6 columns]

## Data Cleaning

### Handle Missing Values

In [4]:
for ticker in tickerList:  # find number of missing rows
    print(f"{ticker}: ")
    print(df.loc[ticker].isnull().sum())
    print()

aapl.us: 
Open       0
High       0
Low        0
Close      0
Volume     0
OpenInt    0
dtype: int64

amzn.us: 
Open       0
High       0
Low        0
Close      0
Volume     0
OpenInt    0
dtype: int64

googl.us: 
Open       0
High       0
Low        0
Close      0
Volume     0
OpenInt    0
dtype: int64

msft.us: 
Open       0
High       0
Low        0
Close      0
Volume     0
OpenInt    0
dtype: int64

tsla.us: 
Open       0
High       0
Low        0
Close      0
Volume     0
OpenInt    0
dtype: int64



In [5]:
condition = df.index.map(lambda index: pd.isna(index[1]))
df.drop(df.index[condition], inplace=True)  # drop rows with NaN date

df.dropna(subset=["Volume", "High", "Low"], inplace=True)  # delete rows with NaN Values

df["Open"] = df["Open"].fillna(df["Close"].shift(1))  # open of day X = close of day (X-1)

df["Close"] = df["Close"].fillna(df["Open"].shift(-1))  # close of day X = open of day (X+1)

df.dropna(subset=["Open", "Close"], inplace=True)   #drop NaN open and close

### Clean data according to dates

In [6]:
df.index = df.index.set_levels(
    pd.to_datetime(df.index.levels[1], errors="coerce"), level=1
)  # convert to datetime format

In [7]:
df.sort_index(inplace=True)  # sort index in each ticker

In [8]:
# keep only last 10years data
# currDate = pd.to_datetime("today")    # 10years from today
currDate = min([df.loc[i].index[-1] for i in tickerList])  # 10yrs from the last data collection date
lastDate = currDate - pd.tseries.offsets.DateOffset(years=10)

condition = df.index.map(lambda index: index[1].year < lastDate.year)
df.drop(df.index[condition], inplace=True)

## Data Transformation

In [9]:
# adding new columns
df["Daily Return"] = ((df["Close"] / df["Open"]) - 1) * 100

# 7 day sliding window grouped on basis of ticker
df["7-Day Moving Average"] = df.groupby(
    level=0, as_index=False, group_keys=False
).apply(lambda x: x.rolling("7D", on=df.index.levels[1])["Close"].mean())

df["30-Day Moving Average"] = df.groupby(
    level=0, as_index=False, group_keys=False
).apply(lambda x: x.rolling("30D", on=df.index.levels[1])["Close"].mean())

# 30day movind standard deviation; requires min 3 values in the sliding window to calulate std (for accuracy purposes)
df["Rolling Volatility (30d)"] = df.groupby(
    level=0, as_index=False, group_keys=False
).apply(
    lambda x: x.rolling("30D", on=df.index.levels[1], min_periods=4)[
        "Daily Return"
    ].std()
)

In [10]:
df

Open     High      Low    Close     Volume  OpenInt  \
        Date                                                                 
aapl.us 2007-01-03   11.049   11.087   10.486   10.731  345302870        0   
        2007-01-04   10.780   11.007   10.733   10.969  236252357        0   
        2007-01-05   10.992   11.038   10.807   10.893  232773093        0   
        2007-01-08   11.011   11.081   10.922   10.946  222149027        0   
        2007-01-09   11.075   11.907   10.906   11.854  933759387        0   
...                     ...      ...      ...      ...        ...      ...   
tsla.us 2017-11-06  307.000  307.500  299.010  302.780    6482486        0   
        2017-11-07  301.020  306.500  300.030  306.050    5286320        0   
        2017-11-08  305.500  306.890  301.300  304.310    4725510        0   
        2017-11-09  302.500  304.460  296.300  302.990    5440335        0   
        2017-11-10  302.500  308.360  301.850  302.990    4621912        0   

                    Daily Return  7-Day Moving Average  30-Day Moving Average  \
        Date                                                                    
aapl.us 2007-01-03     -2.878089             10.731000              10.731000   
        2007-01-04      1.753247             10.850000              10.850000   
        2007-01-05     -0.900655             10.864333              10.864333   
        2007-01-08     -0.590319             10.884750              10.884750   
        2007-01-09      7.033860             11.078600              11.078600   
...                          ...                   ...                    ...   
tsla.us 2017-11-06     -1.374593            307.302500             335.620500   
        2017-11-07      1.670985            307.052000             333.143500   
        2017-11-08     -0.389525            303.698000             330.629000   
        2017-11-09      0.161983            304.444000             327.994500   
        2017-11-10      0.161983            303.824000             324.037368   

                    Rolling Volatility (30d)  
        Date                                  
aapl.us 2007-01-03                       NaN  
        2007-01-04                       NaN  
        2007-01-05                       NaN  
        2007-01-08                  1.897920  
        2007-01-09                  3.810784  
...                                      ...  
tsla.us 2017-11-06                  1.927120  
        2017-11-07                  1.870204  
        2017-11-08                  1.866604  
        2017-11-09                  1.852607  
        2017-11-10                  1.902144  

[12802 rows x 10 columns]

## Exploratory Analysis

In [11]:
ans = (-1e9, 0)
for ticker in tickerList:
    ans = max(
        ans, (df.loc[ticker]["Daily Return"].mean(), ticker)
    )  # find the stock with max avg return over 10 year period

print(
    f"{ans[1]} with an average return of {ans[0]}, had the highest average return over the last 10-year period"
)

amzn.us with an average return of 0.0971722623339749, had the highest average return over the last 10-year period


In [12]:
mxInd = df["Rolling Volatility (30d)"].idxmax()  # get index of highest volatile month
mxVal = df.loc[mxInd][
    "Rolling Volatility (30d)"
]  # get the value corresponding to that index
endDate = mxInd[1]
startDate = endDate - pd.tseries.offsets.DateOffset(months=1)
print(
    f"{mxInd[0]} had the most volatile month; The month starting at {startDate.date()} and ending at {endDate.date()}.\nThe volatility was {mxVal} "
)

tsla.us had the most volatile month; The month starting at 2010-06-01 and ending at 2010-07-01.
The volatility was 16.922146831670265 


## Functions for debugging and checking the df created

In [13]:
# df.tail()
# df.head()
# df.info()
# df.shape()
# df.describe()
# df.loc['tsla.us'].head(10)
# df1.rolling('2D').sum()
# df.head(30)
# df